In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import pickle
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import ElasticNet
from sklearn import model_selection as ms
from scipy.stats import pearsonr
from sklearn.metrics import make_scorer
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold

rand_st=4

In [2]:
output=pd.read_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx')
errors=pd.read_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx')

In [3]:
elos_1=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos1.csv')
elos_2=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos2.csv')
elos_3=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos3.csv')
elos_4=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos4.csv')
features_1=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features1.csv')
features_3=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features3.csv')
features_2=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features2.csv')
features_4=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features4.csv')

In [4]:
X_train_mean_1, X_test_mean_1, Y_train_mean_1, Y_test_mean_1 =ms.train_test_split(features_1, elos_1['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_1, X_test_diff_1, Y_train_diff_1, Y_test_diff_1 =ms.train_test_split(features_1, elos_1['DiffElos'], test_size = 0.3,random_state=rand_st)

X_train_mean_2, X_test_mean_2, Y_train_mean_2, Y_test_mean_2 =ms.train_test_split(features_2, elos_2['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_2, X_test_diff_2, Y_train_diff_2, Y_test_diff_2 =ms.train_test_split(features_2, elos_2['DiffElos'], test_size = 0.3,random_state=rand_st)

X_train_mean_3, X_test_mean_3, Y_train_mean_3, Y_test_mean_3 =ms.train_test_split(features_3, elos_3['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_3, X_test_diff_3, Y_train_diff_3, Y_test_diff_3 =ms.train_test_split(features_3, elos_3['DiffElos'], test_size = 0.3,random_state=rand_st)

X_train_mean_4, X_test_mean_4, Y_train_mean_4, Y_test_mean_4 =ms.train_test_split(features_4, elos_4['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_4, X_test_diff_4, Y_train_diff_4, Y_test_diff_4 =ms.train_test_split(features_4, elos_4['DiffElos'], test_size = 0.3,random_state=rand_st)

In [5]:
model_mean=ElasticNet(alpha=1, copy_X=True, fit_intercept=False, l1_ratio=0,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=4, selection='cyclic', tol=0.0001, warm_start=False)

model_diff=ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=1,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=4, selection='cyclic', tol=0.0001, warm_start=False)

In [6]:
def find_mean_error(pred,elos):
    er=0
    count=0
    for ind, val in enumerate(pred):
        er+=abs(elos.iloc[ind,0]-val[0])
        er+=abs(elos.iloc[ind,1]-val[1])
        count+=2
    return round(er/count,3)

def find_abs_error(pred,elos):
    er=0
    for ind, val in enumerate(pred):
        er+=abs(elos.iloc[ind,0]-val[0])
        er+=abs(elos.iloc[ind,1]-val[1])
    return round(er,3)

def find_root_mean_squared_error(pred,elos):
    er=0
    count=0
    for ind, val in enumerate(pred):
        er+=(elos.iloc[ind,0]-val[0])**2
        er+=(elos.iloc[ind,1]-val[1])**2
        count+=2
    return round((er/count)**0.5,3)

In [7]:
def predictions_report(pred_target,predictions):
    mae_report=mae(pred_target,predictions)
    print ('MAE = {:.3f}'.format(mae_report))
    
    rmse_report = (mse(pred_target, predictions))**0.5
    print ('RMSE = {:.3f}'.format(rmse_report))
    
    corr_coef = pearsonr(pred_target,predictions)
    print("Correlation coefficient = {:.3f}".format(corr_coef[0]))
    
    return (mae_report,rmse_report,corr_coef)

### Подбор модели для 1 кластера

In [8]:
start1 = time.time()

elnet1=model_mean.fit(X_train_mean_1,Y_train_mean_1)
elnet_pred1=elnet1.predict(X_test_mean_1)
scores1=predictions_report(Y_test_mean_1,elnet_pred1)
full_time1=round(time.time() - start1,3)
print(full_time1, 'c')

MAE = 240.279
RMSE = 313.460
Correlation coefficient = 0.147
2.91 c


C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 755706489.5261495, tolerance: 7859553.5027
  positive)


In [9]:
output = output.append(pd.Series([str(model_mean),
                                 scores1[0],scores1[1], 
                                 scores1[2][0],full_time1,'Обучение на среднем ЭЛО по 1 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.744949,217.273742,0.428199,1.174,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.440190,192.130211,0.572216,0.113,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",166.629660,207.827945,0.569104,0.107,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",146.568185,187.970319,0.641182,0.008,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",117.564897,150.552498,0.831412,0.009,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",95.694204,120.219588,0.873861,0.008,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.852117,217.358897,0.509381,0.013,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",149.357283,190.437205,0.594318,0.012,Обучение на разнице в ЭЛО по 4 кластеру
8,"AdaBoostRegressor(base_estimator=None, learnin...",179.646548,226.247606,0.377671,92.545,Обучение на среднем ЭЛО по 1 кластеру
9,"AdaBoostRegressor(base_estimator=None, learnin...",156.774713,200.334686,0.535215,10.463,Обучение на разнице ЭЛО по 1 кластеру


In [10]:
start2 = time.time()

elnet2=model_diff.fit(X_train_diff_1,Y_train_diff_1)
elnet_pred2=elnet2.predict(X_test_mean_1)
scores2=predictions_report(Y_test_diff_1,elnet_pred2)
full_time2=round(time.time() - start2,3)
print(full_time2, 'c')

MAE = 149.852
RMSE = 191.559
Correlation coefficient = 0.561
1.486 c


In [11]:
output = output.append(pd.Series([str(model_diff),
                                 scores2[0],scores2[1], 
                                 scores2[2][0],full_time2,'Обучение на разнице ЭЛО по 1 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.744949,217.273742,0.428199,1.174,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.440190,192.130211,0.572216,0.113,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",166.629660,207.827945,0.569104,0.107,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",146.568185,187.970319,0.641182,0.008,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",117.564897,150.552498,0.831412,0.009,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",95.694204,120.219588,0.873861,0.008,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.852117,217.358897,0.509381,0.013,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",149.357283,190.437205,0.594318,0.012,Обучение на разнице в ЭЛО по 4 кластеру
8,"AdaBoostRegressor(base_estimator=None, learnin...",179.646548,226.247606,0.377671,92.545,Обучение на среднем ЭЛО по 1 кластеру
9,"AdaBoostRegressor(base_estimator=None, learnin...",156.774713,200.334686,0.535215,10.463,Обучение на разнице ЭЛО по 1 кластеру


In [12]:
output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)

In [13]:
white_elos_1 = elnet_pred1 + elnet_pred2/2
black_elos_1 = elnet_pred1-elnet_pred2/2
pred_elos1=list(zip(white_elos_1,black_elos_1))
print(len(pred_elos1))
pred_elos1[:5]

6577


[(2065.4585411722733, 2080.405461053862),
 (2023.7543758819813, 2012.7478551346987),
 (2079.2927877679135, 2274.433474422572),
 (1863.4555516830555, 2014.3237962396893),
 (2241.2764799198126, 2281.357418699908)]

In [14]:
find_abs_error(pred_elos1,elos_1),find_mean_error(pred_elos1,elos_1),find_root_mean_squared_error(pred_elos1,elos_1)

(3676406.928, 279.49, 360.946)

In [15]:
inner_mae_1=find_mean_error(pred_elos1,elos_1)

In [16]:
errors =errors.append(pd.Series(['1 кластер','ElasticNet',
                  find_abs_error(pred_elos1,elos_1),
                  find_mean_error(pred_elos1,elos_1),
                   find_root_mean_squared_error(pred_elos1,elos_1) ],
                                   index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,1 Кластер,LinearRegression,8.306192e+06,189.45700,237.56700
1,2 кластер,LinearRegression,2.481410e+05,180.86100,228.08700
2,3 кластер,LinearRegression,5.772583e+04,126.03900,162.09500
3,4 кластер,LinearRegression,5.350485e+05,188.53000,237.30000
4,Общая ошибка,LinearRegression,2.286777e+06,171.22175,216.26225
5,1 кластер,AdaBoost,3.217283e+06,244.58600,306.16900
6,2 кластер,AdaBoost,2.810322e+05,682.11700,886.27600
7,3 кластер,AdaBoost,5.346101e+04,387.39900,479.34900
8,4 кластер,AdaBoost,3.108843e+05,364.88800,454.87300
9,Общая ошибка,AdaBoost,9.656651e+05,419.74750,531.66675


In [17]:
errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)

### Подбор модели для 2 кластера

In [18]:
start3 = time.time()

elnet3=model_mean.fit(X_train_mean_2,Y_train_mean_2)
elnet_pred3=elnet3.predict(X_test_mean_2)
scores3=predictions_report(Y_test_mean_2,elnet_pred3)
full_time3=round(time.time() - start3,3)
print(full_time3, 'c')

MAE = 238.060
RMSE = 308.318
Correlation coefficient = 0.274
0.063 c


C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 15033274.831352936, tolerance: 225661.2386
  positive)


In [19]:
output = output.append(pd.Series([str(model_mean),
                                 scores3[0],scores3[1], 
                                 scores3[2][0],full_time3,'Обучение на среднем ЭЛО по 2 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.744949,217.273742,0.428199,1.174,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.440190,192.130211,0.572216,0.113,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",166.629660,207.827945,0.569104,0.107,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",146.568185,187.970319,0.641182,0.008,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",117.564897,150.552498,0.831412,0.009,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",95.694204,120.219588,0.873861,0.008,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.852117,217.358897,0.509381,0.013,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",149.357283,190.437205,0.594318,0.012,Обучение на разнице в ЭЛО по 4 кластеру
8,"AdaBoostRegressor(base_estimator=None, learnin...",179.646548,226.247606,0.377671,92.545,Обучение на среднем ЭЛО по 1 кластеру
9,"AdaBoostRegressor(base_estimator=None, learnin...",156.774713,200.334686,0.535215,10.463,Обучение на разнице ЭЛО по 1 кластеру


In [20]:
start4 = time.time()

elnet4=model_diff.fit(X_train_diff_2,Y_train_diff_2)
elnet_pred4=elnet4.predict(X_test_mean_2)
scores4=predictions_report(Y_test_diff_2,elnet_pred4)
full_time4=round(time.time() - start4,3)
print(full_time4, 'c')

MAE = 197.253
RMSE = 278.141
Correlation coefficient = 0.289
0.055 c


C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7876960.682450147, tolerance: 2909.7234647916666
  positive)


In [21]:
output = output.append(pd.Series([str(model_diff),
                                 scores4[0],scores4[1], 
                                 scores4[2][0],full_time4,'Обучение на разнице ЭЛО по 2 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.744949,217.273742,0.428199,1.174,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.440190,192.130211,0.572216,0.113,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",166.629660,207.827945,0.569104,0.107,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",146.568185,187.970319,0.641182,0.008,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",117.564897,150.552498,0.831412,0.009,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",95.694204,120.219588,0.873861,0.008,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.852117,217.358897,0.509381,0.013,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",149.357283,190.437205,0.594318,0.012,Обучение на разнице в ЭЛО по 4 кластеру
8,"AdaBoostRegressor(base_estimator=None, learnin...",179.646548,226.247606,0.377671,92.545,Обучение на среднем ЭЛО по 1 кластеру
9,"AdaBoostRegressor(base_estimator=None, learnin...",156.774713,200.334686,0.535215,10.463,Обучение на разнице ЭЛО по 1 кластеру


In [22]:
output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)

In [23]:
white_elos_2 = elnet_pred3 + elnet_pred3/2
black_elos_2 = elnet_pred3-elnet_pred4/2
pred_elos2=list(zip(white_elos_2,black_elos_2))
print(len(pred_elos2))
pred_elos2[:5]

206


[(3598.2009427661797, 2489.666754137636),
 (3242.7540318731935, 2155.9787561446883),
 (3742.575196096527, 2425.8803932928954),
 (2820.3091414834844, 1822.4178112301017),
 (2812.0330849616284, 1883.5580914441373)]

In [24]:
find_abs_error(pred_elos2,elos_2),find_mean_error(pred_elos2,elos_2),find_root_mean_squared_error(pred_elos2,elos_2)

(279468.933, 678.323, 865.258)

In [25]:
inner_mae_2=find_mean_error(pred_elos2,elos_2)

In [26]:
errors =errors.append(pd.Series(['2 кластер','ElasticNet',
                        find_abs_error(pred_elos2,elos_2),
                        find_mean_error(pred_elos2,elos_2),
                        find_root_mean_squared_error(pred_elos2,elos_2)],
                                index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,1 Кластер,LinearRegression,8.306192e+06,189.45700,237.56700
1,2 кластер,LinearRegression,2.481410e+05,180.86100,228.08700
2,3 кластер,LinearRegression,5.772583e+04,126.03900,162.09500
3,4 кластер,LinearRegression,5.350485e+05,188.53000,237.30000
4,Общая ошибка,LinearRegression,2.286777e+06,171.22175,216.26225
5,1 кластер,AdaBoost,3.217283e+06,244.58600,306.16900
6,2 кластер,AdaBoost,2.810322e+05,682.11700,886.27600
7,3 кластер,AdaBoost,5.346101e+04,387.39900,479.34900
8,4 кластер,AdaBoost,3.108843e+05,364.88800,454.87300
9,Общая ошибка,AdaBoost,9.656651e+05,419.74750,531.66675


### Подбор модели для 3 кластера

In [27]:
start5 = time.time()

elnet5=model_mean.fit(X_train_mean_3,Y_train_mean_3)
elnet_pred5=elnet5.predict(X_test_mean_3)
scores5=predictions_report(Y_test_mean_3,elnet_pred5)
full_time5=round(time.time() - start5,3)
print(full_time5, 'c')

MAE = 2558.475
RMSE = 18692.100
Correlation coefficient = -0.070
0.024 c


C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2115864.0490463595, tolerance: 72126.4739
  positive)


In [28]:
output = output.append(pd.Series([str(model_mean),
                                 scores5[0],scores5[1], 
                                 scores5[2][0],full_time5,'Обучение на среднем ЭЛО по 3 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.744949,217.273742,0.428199,1.174,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.440190,192.130211,0.572216,0.113,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",166.629660,207.827945,0.569104,0.107,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",146.568185,187.970319,0.641182,0.008,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",117.564897,150.552498,0.831412,0.009,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",95.694204,120.219588,0.873861,0.008,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.852117,217.358897,0.509381,0.013,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",149.357283,190.437205,0.594318,0.012,Обучение на разнице в ЭЛО по 4 кластеру
8,"AdaBoostRegressor(base_estimator=None, learnin...",179.646548,226.247606,0.377671,92.545,Обучение на среднем ЭЛО по 1 кластеру
9,"AdaBoostRegressor(base_estimator=None, learnin...",156.774713,200.334686,0.535215,10.463,Обучение на разнице ЭЛО по 1 кластеру


In [29]:
start6 = time.time()

elnet6=model_diff.fit(X_train_diff_3,Y_train_diff_3)
elnet_pred6=elnet6.predict(X_test_mean_3)
scores6=predictions_report(Y_test_diff_3,elnet_pred6)
full_time6=round(time.time() - start6,3)
print(full_time6, 'c')

MAE = 10415.503
RMSE = 83063.928
Correlation coefficient = -0.156
0.023 c


C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 670161.370637828, tolerance: 970.1791975000002
  positive)


In [30]:
output = output.append(pd.Series([str(model_diff),
                                 scores6[0],scores6[1], 
                                 scores6[2][0],full_time6,'Обучение на разнице ЭЛО по 3 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.744949,217.273742,0.428199,1.174,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.440190,192.130211,0.572216,0.113,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",166.629660,207.827945,0.569104,0.107,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",146.568185,187.970319,0.641182,0.008,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",117.564897,150.552498,0.831412,0.009,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",95.694204,120.219588,0.873861,0.008,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.852117,217.358897,0.509381,0.013,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",149.357283,190.437205,0.594318,0.012,Обучение на разнице в ЭЛО по 4 кластеру
8,"AdaBoostRegressor(base_estimator=None, learnin...",179.646548,226.247606,0.377671,92.545,Обучение на среднем ЭЛО по 1 кластеру
9,"AdaBoostRegressor(base_estimator=None, learnin...",156.774713,200.334686,0.535215,10.463,Обучение на разнице ЭЛО по 1 кластеру


In [31]:
output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)

In [32]:
white_elos_3 = elnet_pred5 + elnet_pred6/2
black_elos_3 = elnet_pred5-elnet_pred6/2
pred_elos3=list(zip(white_elos_3,black_elos_3))
print(len(pred_elos3))
pred_elos3[:5]

69


[(2281.5238698422027, 1947.062737208712),
 (2178.0805643749936, 2132.7280194003743),
 (1427.4385824879964, 2756.7555939490876),
 (2361.61378719404, 1989.6017628174966),
 (2072.1385159481115, 2233.7162737876215)]

In [33]:
find_abs_error(pred_elos3,elos_3),find_mean_error(pred_elos3,elos_3),find_root_mean_squared_error(pred_elos3,elos_3)

(745366.315, 5401.205, 45543.509)

In [34]:
inner_mae_3=find_mean_error(pred_elos3,elos_3)

In [35]:
errors =errors.append(pd.Series(['3 кластер','ElasticNet',
                                 find_abs_error(pred_elos3,elos_3),
                                 find_mean_error(pred_elos3,elos_3),
                                 find_root_mean_squared_error(pred_elos3,elos_3)],
                                index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,1 Кластер,LinearRegression,8.306192e+06,189.45700,237.56700
1,2 кластер,LinearRegression,2.481410e+05,180.86100,228.08700
2,3 кластер,LinearRegression,5.772583e+04,126.03900,162.09500
3,4 кластер,LinearRegression,5.350485e+05,188.53000,237.30000
4,Общая ошибка,LinearRegression,2.286777e+06,171.22175,216.26225
5,1 кластер,AdaBoost,3.217283e+06,244.58600,306.16900
6,2 кластер,AdaBoost,2.810322e+05,682.11700,886.27600
7,3 кластер,AdaBoost,5.346101e+04,387.39900,479.34900
8,4 кластер,AdaBoost,3.108843e+05,364.88800,454.87300
9,Общая ошибка,AdaBoost,9.656651e+05,419.74750,531.66675


### Подбор модели для 4 кластера

In [36]:
start7 = time.time()

elnet7=model_mean.fit(X_train_mean_4,Y_train_mean_4)
elnet_pred7=elnet7.predict(X_test_mean_4)
scores7=predictions_report(Y_test_mean_4,elnet_pred7)
full_time7=round(time.time() - start7,3)
print(full_time7, 'c')

MAE = 297.102
RMSE = 1358.799
Correlation coefficient = 0.117
0.091 c


C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 39198588.24030254, tolerance: 485907.22770000005
  positive)


In [37]:
output = output.append(pd.Series([str(model_mean),
                                 scores7[0],scores7[1], 
                                 scores7[2][0],full_time7,'Обучение на среднем ЭЛО по 4 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.744949,217.273742,0.428199,1.174,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.440190,192.130211,0.572216,0.113,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",166.629660,207.827945,0.569104,0.107,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",146.568185,187.970319,0.641182,0.008,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",117.564897,150.552498,0.831412,0.009,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",95.694204,120.219588,0.873861,0.008,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.852117,217.358897,0.509381,0.013,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",149.357283,190.437205,0.594318,0.012,Обучение на разнице в ЭЛО по 4 кластеру
8,"AdaBoostRegressor(base_estimator=None, learnin...",179.646548,226.247606,0.377671,92.545,Обучение на среднем ЭЛО по 1 кластеру
9,"AdaBoostRegressor(base_estimator=None, learnin...",156.774713,200.334686,0.535215,10.463,Обучение на разнице ЭЛО по 1 кластеру


In [38]:
start8 = time.time()

elnet8=model_diff.fit(X_train_diff_4,Y_train_diff_4)
elnet_pred8=elnet8.predict(X_test_mean_4)
scores8=predictions_report(Y_test_diff_4,elnet_pred8)
full_time8=round(time.time() - start8,3)
print(full_time8, 'c')

MAE = 196.344
RMSE = 662.446
Correlation coefficient = 0.068
0.102 c


C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 16822442.360465374, tolerance: 5669.037487210474
  positive)


In [39]:
output = output.append(pd.Series([str(model_diff),
                                 scores8[0],scores8[1], 
                                 scores8[2][0],full_time8,'Обучение на разнице ЭЛО по 4 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.744949,217.273742,0.428199,1.174,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.440190,192.130211,0.572216,0.113,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",166.629660,207.827945,0.569104,0.107,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",146.568185,187.970319,0.641182,0.008,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",117.564897,150.552498,0.831412,0.009,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",95.694204,120.219588,0.873861,0.008,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.852117,217.358897,0.509381,0.013,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",149.357283,190.437205,0.594318,0.012,Обучение на разнице в ЭЛО по 4 кластеру
8,"AdaBoostRegressor(base_estimator=None, learnin...",179.646548,226.247606,0.377671,92.545,Обучение на среднем ЭЛО по 1 кластеру
9,"AdaBoostRegressor(base_estimator=None, learnin...",156.774713,200.334686,0.535215,10.463,Обучение на разнице ЭЛО по 1 кластеру


In [40]:
white_elos_4 = elnet_pred7 + elnet_pred8/2
black_elos_4 = elnet_pred7-elnet_pred8/2
pred_elos4=list(zip(white_elos_4,black_elos_4))
print(len(pred_elos4))
pred_elos4[:5]

426


[(1794.691638045343, 1863.8694422555882),
 (2111.510606433196, 2182.0415640648835),
 (2319.9597506806886, 2098.115674761534),
 (2265.5075521612257, 2312.1183715563734),
 (2129.380565086777, 2049.103849434774)]

In [41]:
find_abs_error(pred_elos4,elos_4),find_mean_error(pred_elos4,elos_4),find_root_mean_squared_error(pred_elos4,elos_4)

(323397.602, 379.575, 1427.39)

In [42]:
inner_mae_4=find_mean_error(pred_elos4,elos_4)

In [43]:
errors =errors.append(pd.Series(['4 кластер','ElasticNet',find_abs_error(pred_elos4,elos_4),
                                 find_mean_error(pred_elos4,elos_4),
                                 find_root_mean_squared_error(pred_elos4,elos_4)],
                                index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,1 Кластер,LinearRegression,8.306192e+06,189.45700,237.56700
1,2 кластер,LinearRegression,2.481410e+05,180.86100,228.08700
2,3 кластер,LinearRegression,5.772583e+04,126.03900,162.09500
3,4 кластер,LinearRegression,5.350485e+05,188.53000,237.30000
4,Общая ошибка,LinearRegression,2.286777e+06,171.22175,216.26225
5,1 кластер,AdaBoost,3.217283e+06,244.58600,306.16900
6,2 кластер,AdaBoost,2.810322e+05,682.11700,886.27600
7,3 кластер,AdaBoost,5.346101e+04,387.39900,479.34900
8,4 кластер,AdaBoost,3.108843e+05,364.88800,454.87300
9,Общая ошибка,AdaBoost,9.656651e+05,419.74750,531.66675


In [44]:
elnet_mae=(inner_mae_4+inner_mae_3+inner_mae_2+inner_mae_1)/4
elnet_mae

1684.64825

In [45]:
elnet_abs=(find_abs_error(pred_elos1,elos_1)+
        find_abs_error(pred_elos2,elos_2)+
        find_abs_error(pred_elos3,elos_3)+
        find_abs_error(pred_elos4,elos_4))/4
elnet_abs

1256159.9445

In [46]:
elnet_rmse=(find_root_mean_squared_error(pred_elos1,elos_1)+
         find_root_mean_squared_error(pred_elos2,elos_2)+
         find_root_mean_squared_error(pred_elos3,elos_3)+
         find_root_mean_squared_error(pred_elos4,elos_4))/4
elnet_rmse

12049.275749999999

In [47]:
errors = errors.append(pd.Series(['Общая ошибка','ElasticNet',
                  elnet_abs,
                  elnet_mae,
                 elnet_rmse],index=errors.columns) ,ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,1 Кластер,LinearRegression,8.306192e+06,189.45700,237.56700
1,2 кластер,LinearRegression,2.481410e+05,180.86100,228.08700
2,3 кластер,LinearRegression,5.772583e+04,126.03900,162.09500
3,4 кластер,LinearRegression,5.350485e+05,188.53000,237.30000
4,Общая ошибка,LinearRegression,2.286777e+06,171.22175,216.26225
5,1 кластер,AdaBoost,3.217283e+06,244.58600,306.16900
6,2 кластер,AdaBoost,2.810322e+05,682.11700,886.27600
7,3 кластер,AdaBoost,5.346101e+04,387.39900,479.34900
8,4 кластер,AdaBoost,3.108843e+05,364.88800,454.87300
9,Общая ошибка,AdaBoost,9.656651e+05,419.74750,531.66675
